# RAG Evaluation

## Load Dependencies

In [ ]:
%pip install azure-ai-evaluation
%pip install promptflow-azure

## Load Azure configurations

You always need to run this!

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv() # take environment variables from .env.

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")

model_config = {
    "azure_endpoint": azure_openai_endpoint,
    "api_key": azure_openai_key,
    "azure_deployment": azure_openai_deployment,
}

azure_subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
azure_resource_group_name = os.getenv("AZURE_RESOURCE_GROUP_NAME")
azure_project_name = os.getenv("AZURE_PROJECT_NAME")

azure_ai_project = {
    "subscription_id": azure_subscription_id,
    "resource_group_name": azure_resource_group_name,
    "project_name": azure_project_name,
}

## Get the first row to test

In [ ]:
import json

# Load JSON data from a file
with open('data/output/eval.jsonl', 'r') as file:
    data = [json.loads(line) for line in file]

# Assuming the JSON structure is a list of dictionaries and we want the first row
first_row = data[0]

# Assign values to variables
context = first_row['context']
query = first_row['query']
ground_truth = first_row['ground_truth']
response = first_row['response']

## Performance Evaluators

In [ ]:
from azure.ai.evaluation import GroundednessEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator
from azure.ai.evaluation import RougeScoreEvaluator, RougeType
from azure.ai.evaluation import BleuScoreEvaluator
from azure.ai.evaluation import MeteorScoreEvaluator
from azure.ai.evaluation import GleuScoreEvaluator

groundedness_eval = GroundednessEvaluator(model_config)
groundedness_score = groundedness_eval(
    response=response,
    context=context,
)

relevance_eval = RelevanceEvaluator(model_config)
relevance_score = relevance_eval(
    response=response,
    context=context,
    query=query
)

coherence_eval = CoherenceEvaluator(model_config)
coherence_score = coherence_eval(
    response=response,
    query=query
)

fluency_eval = FluencyEvaluator(model_config)
fluency_score = fluency_eval(
    response=response,
    query=query
)

similarity_eval = SimilarityEvaluator(model_config)
similarity_score = similarity_eval(
    response=response,
    query=query,
    ground_truth=ground_truth
)

f1_eval = F1ScoreEvaluator()
f1_score = f1_eval(
    response=response,
    ground_truth=ground_truth
)

# There are several types of ROUGE metrics: ROUGE_1, ROUGE_2, ROUGE_3, ROUGE_4, ROUGE_5, and ROUGE_L.
rouge_eval = RougeScoreEvaluator(rouge_type=RougeType.ROUGE_1)
rouge_score = rouge_eval(
    response=response,
    ground_truth=ground_truth,
)

bleu_eval = BleuScoreEvaluator()
bleu_score = bleu_eval(
    response=response,
    ground_truth=ground_truth
)

meteor_eval = MeteorScoreEvaluator(
    alpha=0.9,
    beta=3.0,
    gamma=0.5
)
meteor_score = meteor_eval(
    response=response,
    ground_truth=ground_truth,
)

gleu_eval = GleuScoreEvaluator()
gleu_score = gleu_eval(
    response=response,
    ground_truth=ground_truth,
)

print(groundedness_score)
print(relevance_score)
print(coherence_score)
print(fluency_score)
print(similarity_score)
print(f1_score)
print(rouge_score)
print(bleu_score)
print(meteor_score)
print(gleu_score)

## Risk and Safety Metrics

1. Install Azure CLI in Github Codespaces
- curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

2. Login with your Azure account 
- az login --use-device-code
- Once you've logged in, select your subscription in the terminal.

In [ ]:
from azure.ai.evaluation import ViolenceEvaluator, HateUnfairnessEvaluator, SelfHarmEvaluator,SexualEvaluator
from azure.identity import DefaultAzureCredential

violence_eval = ViolenceEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
violence_score = violence_eval(query=query, response=response)

hateunfairness_eval = HateUnfairnessEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
hateunfairness_score = hateunfairness_eval(query=query, response=response)

selfharm_eval = SelfHarmEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
selfharm_score = selfharm_eval(query=query, response=response)

sexual_eval = SexualEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
sexual_score = sexual_eval(query=query, response=response)

print(violence_score)
print(hateunfairness_score)
print(selfharm_score)
print(sexual_score)

## Evaluate test dataset

In [ ]:
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import GroundednessEvaluator, RetrievalEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator
from azure.ai.evaluation import RougeScoreEvaluator, RougeType
from azure.ai.evaluation import BleuScoreEvaluator
from azure.ai.evaluation import MeteorScoreEvaluator
from azure.ai.evaluation import GleuScoreEvaluator
from azure.ai.evaluation import ViolenceEvaluator, HateUnfairnessEvaluator, SelfHarmEvaluator,SexualEvaluator
from azure.identity import DefaultAzureCredential
import pandas as pd

groundedness_eval = GroundednessEvaluator(model_config)
retrieval_eval = RetrievalEvaluator(model_config)
relevance_eval = RelevanceEvaluator(model_config)
coherence_eval = CoherenceEvaluator(model_config)
fluency_eval = FluencyEvaluator(model_config)
similarity_eval = SimilarityEvaluator(model_config)
f1_eval = F1ScoreEvaluator()
rouge_eval = RougeScoreEvaluator(rouge_type=RougeType.ROUGE_1)
bleu_eval = BleuScoreEvaluator()
meteor_eval = MeteorScoreEvaluator(
    alpha=0.9,
    beta=3.0,
    gamma=0.5
)
gleu_eval = GleuScoreEvaluator()
violence_eval = ViolenceEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
hateunfairness_eval = HateUnfairnessEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
selfharm_eval = SelfHarmEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
sexual_eval = SexualEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())

path = "data/output/eval.jsonl"

result = evaluate(
    data=path, # provide your data here
    evaluators={
        "groundedness": groundedness_eval,
        "retrieval": retrieval_eval,
        "relevance": relevance_eval,
        "coherence": coherence_eval,
        "fluency": fluency_eval,
        "similarity":similarity_eval,
        "f1_score": f1_eval,
        "rouge_score": rouge_eval,
        "bleu_score": bleu_eval,
        "meteor_score": meteor_eval,
        "gleu_score": gleu_eval,
        "violence_score": violence_eval,
        "hateunfairness_score": hateunfairness_eval,
        "selfharm_score": selfharm_eval,
        "sexual_score": sexual_eval         
    },
    # column mapping
    evaluator_config={
        "default": {
            "query": "${data.query}",
            "response": "${data.response}",
            "context": "${data.context}",
            "ground_truth": "${data.ground_truth}"
        }
    }
)

df = pd.DataFrame(result["rows"])
# Save the DataFrame to a CSV file
df.to_csv('data/output/evalresult.csv', index=False)

print("DataFrame has been successfully saved to evalresult.csv")

## Assign yourself the Proper role to Track results in Azure AI Foundry

1. Get your user ID

az ad signed-in-user show --query id --output tsv

2. Assign yourself the Storage Blob Data Contributor role. Replace the placeholder text with your subscription ID, resource group, and user ID.

az role assignment create --role "Storage Blob Data Contributor" --scope /subscriptions/mySubscriptionID/resourceGroups/myResourceGroupName --assignee-principal-type User --assignee-object-id "user-id"

## Run Evaluation and Track in Azure AI Foundry

In [ ]:
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import GroundednessEvaluator, RetrievalEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator
from azure.ai.evaluation import RougeScoreEvaluator, RougeType
from azure.ai.evaluation import BleuScoreEvaluator
from azure.ai.evaluation import MeteorScoreEvaluator
from azure.ai.evaluation import GleuScoreEvaluator
from azure.ai.evaluation import ViolenceEvaluator, HateUnfairnessEvaluator, SelfHarmEvaluator,SexualEvaluator
from azure.identity import DefaultAzureCredential
import pandas as pd

groundedness_eval = GroundednessEvaluator(model_config)
retrieval_eval = RetrievalEvaluator(model_config)
relevance_eval = RelevanceEvaluator(model_config)
coherence_eval = CoherenceEvaluator(model_config)
fluency_eval = FluencyEvaluator(model_config)
similarity_eval = SimilarityEvaluator(model_config)
f1_eval = F1ScoreEvaluator()
rouge_eval = RougeScoreEvaluator(rouge_type=RougeType.ROUGE_1)
bleu_eval = BleuScoreEvaluator()
meteor_eval = MeteorScoreEvaluator(
    alpha=0.9,
    beta=3.0,
    gamma=0.5
)
gleu_eval = GleuScoreEvaluator()
violence_eval = ViolenceEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
hateunfairness_eval = HateUnfairnessEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
selfharm_eval = SelfHarmEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
sexual_eval = SexualEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())

path = "data/output/eval.jsonl"

result = evaluate(
    data=path, # provide your data here
    evaluators={
        "groundedness": groundedness_eval,
        "retrieval": retrieval_eval,
        "relevance": relevance_eval,
        "coherence": coherence_eval,
        "fluency": fluency_eval,
        "similarity": similarity_eval,
        "f1_score": f1_eval,
        "rouge_score": rouge_eval,
        "bleu_score": bleu_eval,
        "meteor_score": meteor_eval,
        "gleu_score": gleu_eval,
        "violence_score": violence_eval,
        "hateunfairness_score": hateunfairness_eval,
        "selfharm_score": selfharm_eval,
        "sexual_score": sexual_eval 
    },
    # column mapping
    evaluator_config={
        "default": {
            "query": "${data.query}",
            "response": "${data.response}",
            "context": "${data.context}",
            "ground_truth": "${data.ground_truth}"
        }
    },
    azure_ai_project = azure_ai_project
)


## View Evaluation Results

In [ ]:
print(result['studio_url'])

## Evaluate Using a Custom Evaluator

In [ ]:
from promptflow.client import load_flow

friendliness_eval = load_flow(source="friendliness.prompty", model={"configuration": model_config})
friendliness_score = friendliness_eval(
    query=query,
    response=response
)
print(friendliness_score)